In [1]:
import os

%pwd

os.chdir("../")
%pwd

'/Users/harendrakumar/Documents/Demand_forecast'

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import os
from typing import Dict, Any, Tuple
from pathlib import Path
import joblib

from dataclasses import dataclass

In [3]:
from Demand_Forecast.constants import *
from Demand_Forecast.utils.common import read_yaml, create_directories

In [4]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    order: Tuple[int, int, int]
    seasonal_order: Tuple[int, int, int, int]
    days_in_future: int
    initial_inventory: int
    lead_time: int
    service_level: float
    holding_cost: float
    stockout_cost: int
    target_column: str
    

In [5]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_roots])
    
    def get_model_trainer(self):
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN
        params = self.params.Inventory
        fcst_parms = self.params.Forecast

        create_directories([config.root_dir])
        return ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            order= fcst_parms.order,
            seasonal_order= fcst_parms.seasonal_order,
            days_in_future = fcst_parms.days_in_future,
            initial_inventory= params.initial_inventory,
            lead_time= params.lead_time,
            service_level= params.service_level,
            holding_cost= params.holding_cost,
            stockout_cost= params.stockout_cost,
            target_column= schema.name
        )

In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    
    @staticmethod
    def parse_tuple(input):
        return tuple(int(x) for x in input.strip('()').split(','))
    
    def predictions(self):
        train = pd.read_csv(self.config.train_data_path)
        test =  pd.read_csv(self.config.test_data_path)

        train['Date'] = pd.to_datetime(train['Date'])
        time_series = train.set_index('Date')['Demand']
        order = self.parse_tuple(input=self.config.order)
        seasonal_order = self.parse_tuple(input=self.config.seasonal_order)
        model = SARIMAX(time_series, order= order, seasonal_order= seasonal_order)
        model_fit = model.fit(disp=False)
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

        future_steps = self.config.days_in_future
        predictions = model_fit.predict(len(time_series), len(time_series) + future_steps -1)
        predictions = predictions.astype(int)
        return predictions
    
    def inventory_manage(self) -> Dict[str, int]:
        df = pd.read_csv(self.config.train_data_path)
        df['Date'] = pd.to_datetime(df['Date'])
        time_series = df.set_index('Date')['Demand']
        predictions = self.predictions().to_list()
        
        future_dates = pd.date_range(start=time_series.index[-1] + pd.DateOffset(days=1), periods=self.config.days_in_future, freq='D')
        forecasted_demand = pd.Series(predictions, index=future_dates)
        # Initial Inventory level
        initial_inventory = self.config.initial_inventory
        # Lead time (No of days to replenish inventory)
        lead_time = self.config.lead_time # it's different for every business, 1 is an example
        # Service level (probability of not stocking out)
        service_level = self.config.service_level # it's different for every business, 0.95 is an
        # Calculate the optimal order quantity using the Newsvendor formula
        z = np.abs(np.percentile(forecasted_demand, 100 * (1 - service_level)))
        order_quantity = np.ceil(forecasted_demand.mean() + z).astype(int)
        # Calculate the reorder point
        reorder_point = forecasted_demand.mean() * lead_time + z
        # Calculate the optimal safety stock
        safety_stock = reorder_point - forecasted_demand.mean() * lead_time
        # Calculate the total cost (holding cost + stockout cost)
        holding_cost = self.config.holding_cost  # it's different for every business, 0.1 is an example
        stockout_cost = self.config.stockout_cost  # # it's different for every business, 10 is an example
        total_holding_cost = holding_cost * (initial_inventory + 0.5 * order_quantity)
        total_stockout_cost = stockout_cost * np.maximum(0, forecasted_demand.mean() * lead_time - initial_inventory)

        # Calculate the total cost
        total_cost = total_holding_cost + total_stockout_cost
        return {
            "Optimal Order Quantity" : order_quantity,
            "Reorder Point": reorder_point,
            "Safety Stock": safety_stock,
            "Total Cost": total_cost
        }
        

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer()
    model_trainer = ModelTrainer(config=model_trainer_config)
    predictions = model_trainer.predictions()
    inventory_info = model_trainer.inventory_manage()
    print(inventory_info)
except Exception as e:
    raise e

[2023-10-20 21:03:03,053: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-20 21:03:03,056: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-20 21:03:03,058: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-20 21:03:03,059: INFO: common: created directory at: artifacts]
[2023-10-20 21:03:03,060: INFO: common: created directory at: artifacts/model_trainer]
{'Optimal Order Quantity': 250, 'Reorder Point': 249.95, 'Safety Stock': 118.85, 'Total Cost': 562.5}


/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: Va

In [ ]:
predictions